# Step 3. Baseline Model

[//]: # (TODO: Write methodology and observations here)

In [ ]:
# Imports and environmental setups

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# Custom utils functions
from utils import dataset
from utils.visualization import plot_learning_curve
from utils.visualization import visualize_predictions


plt.style.use('ggplot')
dataset_dir = '../data/processed/'
log_dir = '../log/baseline/'

## 3.1. Load Dataset

In [ ]:
df = dataset.load(dataset_dir)

In [ ]:
df.info()

In [ ]:
df.sample(n=42, random_state=42)

In [ ]:
train, test = train_test_split(df, shuffle=True, test_size=0.2, random_state=42)
train, val = train_test_split(train, shuffle=True, test_size=0.25, random_state=42)

print(f'Train data: {train.shape[0]} samples, Validation Data: {val.shape[0]} samples, Test Data: {test.shape[0]} samples.')

## 3.2. Baseline Model Setup and Training

### 3.2.1. Baseline Parameters

In [ ]:
# Model Parameters
IMG_SIZE = 256
BATCH_SIZE = 512
INPUT_DIM = (IMG_SIZE, IMG_SIZE, 3)  # RGB - 3 channels images
HIDDEN_LAYER_DIM = 128
OUTPUT_CLASSES = 8  # One-hot encoded: 8 different classes

# Training Parameters
EPOCHS = 16
LEARNING_RATE = 1e-4

### 3.2.2. Model Setup

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=INPUT_DIM),
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='relu'),
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='relu'),
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='relu'),
    tf.keras.layers.Dense(HIDDEN_LAYER_DIM, activation='sigmoid'),
    tf.keras.layers.Dense(OUTPUT_CLASSES),
], name='baseline')
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, to_file='images/BaselineModel-Diagram.png')

### 3.2.3. Training

In [ ]:
train['Class'] = train['Class'].astype('str')
train_datagen = ImageDataGenerator(data_format='channels_last')
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=dataset_dir,
    x_col='ImgPath',
    y_col='Class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

val['Class'] = val['Class'].astype('str')
val_datagen = ImageDataGenerator(data_format='channels_last')
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory=dataset_dir,
    x_col='ImgPath',
    y_col='Class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy'],
)

history = model.fit(
    train_generator, validation_data=val_generator,
    epochs=EPOCHS,
    verbose=2, callbacks=[tensorboard_callback]
)

In [ ]:
plot_learning_curve(
    history.history['loss'],
    history.history['val_loss'],
    history.history['categorical_accuracy'],
    history.history['val_categorical_accuracy'],
    to_file='images/BaselineModel-LearningCurve.png'
)

## 3.3. Baseline Model Performance

In [ ]:
test['Class'] = test['Class'].astype('str')
test_datagen = ImageDataGenerator(data_format='channels_last')
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=dataset_dir,
    x_col='ImgPath',
    y_col='Class',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='categorical',
    seed=42
)

model.evaluate(test_generator)

In [ ]:
visualize_predictions(model, test_generator, to_file='images/BaselineModel-SamplePredictions.png')